In [57]:
import pandas as pd
import numpy as np
import string
import re
import jieba
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
from _utils import u_constant
path = u_constant.PATH_ROOT + "for learn/Python/NLP_in_Action/chapter-9/cluster/"

In [14]:
book_data = pd.read_csv(path + "data/data.csv")
book_data[:5]

,title,tag,info,comments,content
0,﻿解忧杂货店,豆瓣图书标签: 小说,[日] 东野圭吾 / 李盈春 / 南海出版公司 / 2014-5 / 39.50元,(225675人评价),现代人内心流失的东西，这家杂货店能帮你找回——\r\n僻静的街道旁有一家杂货店，只要写下烦恼...
1,巨人的陨落,豆瓣图书标签: 小说,[英] 肯·福莱特 / 于大卫 / 江苏凤凰文艺出版社 / 2016-5-1 / 129.80元,(22536人评价),在第一次世界大战的硝烟中，每一个迈向死亡的生命都在热烈地生长——威尔士的矿工少年、刚失恋的美...
2,我的前半生,豆瓣图书标签: 小说,亦舒 / 新世界出版社 / 2007-8 / 22.00元,(20641人评价),一个三十几岁的美丽女人子君，在家做全职家庭主妇。却被一个平凡女人夺走丈夫，一段婚姻的失败，让...
3,百年孤独,豆瓣图书标签: 小说,[哥伦比亚] 加西亚·马尔克斯 / 范晔 / 南海出版公司 / 2011-6 / 39.50元,(111883人评价),《百年孤独》是魔幻现实主义文学的代表作，描写了布恩迪亚家族七代人的传奇故事，以及加勒比海沿岸...
4,追风筝的人,豆瓣图书标签: 小说,[美] 卡勒德·胡赛尼 / 李继宏 / 上海人民出版社 / 2006-5 / 29.00元,(278905人评价),12岁的阿富汗富家少爷阿米尔与仆人哈桑情同手足。然而，在一场风筝比赛后，发生了一件悲惨不堪的...


In [10]:
class Treat:
    
    def __init__(self, feature_extract_method="tfidf", **params):
        self.stopword_list = self.__load_stop_word()
        self.invalid_pattern = re.compile("[{}]".format(re.escape(string.punctuation)))
        self.extract_method = self.__identify_method(feature_extract_method.lower())
        self.params = params
    
    def __identify_method(self, s):
        """
        识别抽取方法
        """
        if s in ["tfidf", "tf-idf", "tf_idf"]:
            return "tfidf"
        return "bow"
        
    def __load_stop_word(self):
        with open(path + "dict/stop_words.utf8", encoding="utf-8") as f:
            stopword_list = f.readlines()
            f.close()
        return stopword_list
    
    def preprocess(self, text):
        # 去除特殊符号
        text = self.invalid_pattern.sub("", text)
        # 分词 & 去停用词
        tokens = [token.strip() for token in jieba.cut(text) \
                  if token.strip() not in self.stopword_list]
        return " ".join(tokens)
    
    def fit_transform(self, corpus):
        """
        处理输入的语料，输出向量化矩阵
        """
        normed_data = list(map(self.preprocess, corpus))
        if self.extract_method == "bow":
            self.vec = CountVectorizer(**self.params)
        elif self.extract_method == "tfidf":
            self.vec = TfidfVectorizer(**self.params)
        features = self.vec.fit_transform(normed_data)
        return features
    
    def transform(self, corpus):
        normed_data = list(map(self.preprocess, corpus))
        return self.vec.transform(normed_data)

In [44]:
treat = Treat(feature_extract_method="tfidf", min_df=0.02, max_df=0.9, ngram_range=(1, 2))

In [48]:
feature_matrix = treat.fit_transform(book_data["content"].values)
feature_names = treat.vec.get_feature_names()
titles = book_data["title"].values
print(feature_matrix.shape)
print(feature_names[:10])

(2822, 93)
['20', '一个', '一本', '一生', '一种', '一部', '不是', '世界', '世纪', '中国']


In [70]:
class cluster:
    
    def __init__(self, n_clusters=10, topn_features=10):
        self.n_clusters = n_clusters
        self.topn_features = topn_features
    
    def fit(self, feature_matrix, titles, feature_names):
        self.titles = titles
        self.feature_names = feature_names
        self.model = KMeans(n_clusters=self.n_clusters, max_iter=10000)
        self.model.fit(feature_matrix)
        self.clusters = self.model.labels_
        
    def get_cluster_data(self):
        """
        输出簇的信息
        """
        cluster_centers = self.model.cluster_centers_
        # 以簇中心的特征（维度）坐标作为特征重要度，这里进行降序，以便获得头部特征
        ordered_centroids = cluster_centers.argsort()[:,::-1]

        cluster_details = defaultdict(dict)
        for cluster, title in zip(self.clusters, self.titles):
            try:
                cluster_details[cluster]["books"].append(title)
            except KeyError:
                cluster_details[cluster]["books"] = [title]
        
        for cluster in cluster_details.keys():
            key_features = [self.feature_names[x]\
                            for x in ordered_centroids[cluster, :self.topn_features]]
            cluster_details[cluster]["key_features"] = key_features
        return dict(cluster_details)

In [71]:
clu = cluster(n_clusters=10, topn_features=5)
clu.fit(feature_matrix, titles, feature_names)
cluster_details = clu.get_cluster_data()

for cluster, cluster_info in cluster_details.items():
    print("Cluster %d details:" % cluster)
    print("--------------------------")
    print("Key features: ", cluster_info["key_features"])
    print("Books in cluster: \n", ",".join(cluster_info["books"]))
    print("==========================")

Cluster 0 details:
--------------------------
Key features:  ['作品', '一部', '一本', '历史', '人类']
Books in cluster: 
 ﻿解忧杂货店,新名字的故事,鱼王,霍乱时期的爱情,活着,杀死一只知更鸟,双峰: 神秘史,新名字的故事,二手时间,鱼王,外婆的道歉信,霍乱时期的爱情,杀死一只知更鸟,我的天才女友,二手时间,鱼王,外婆的道歉信,霍乱时期的爱情,活着,杀死一只知更鸟,双峰: 神秘史,﻿沉默的大多数,目送,我为你洒下月光,吃鲷鱼让我打嗝,当我谈跑步时我谈些什么,孤独六讲,﻿活着,沉默的大多数,爱你就像爱生命,我为你洒下月光,艺术的故事,杀死一只知更鸟,冷暴力,恋情的终结,山海经全译,國史大綱（上下）,经济学原理（上下）,﻿解忧杂货店,百鬼夜行 阳,金色梦乡,强风吹拂,1Q84 BOOK 1,目送,我为你洒下月光,此生多珍重,一个人的村庄,冬牧场,所谓好玩的事，我再也不做了,1Q84 BOOK 1,当我谈跑步时我谈些什么,1Q84 BOOK 2,大萝卜和难挑的鳄梨,国境以南 太阳以西,远方的鼓声,舞！舞！舞！,没有女人的男人们,1Q84 BOOK 3,东京奇谭集,万物静默如谜,我的孤独是一座花园,博尔赫斯诗选,唯有孤独恒常如新,恶之花,夜莺与玫瑰,猜猜我有多爱你,牧羊少年奇幻之旅,银河铁道之夜,安吉拉·卡特的精怪故事集,狐狸的窗户,﻿沉默的大多数,爱你就像爱生命,白银时代,假如你愿意你就恋爱吧,万寿寺,似水流年,三十而立,﻿沉默的大多数,常识,门萨的娼妓,幻想图书馆,写在人生边上 人生边上的边上 石语,普通读者,猜猜我有多爱你,《噼里啪啦系列》,山海经全译,诗经,古文观止,闲情偶寄,东京梦华录,声律启蒙,既见君子,傲慢与偏见,悲惨世界（上中下）,﻿活着,兄弟（上）,现实一种,兄弟（下）,音乐影响了我的写作,余华作品系列（共12册）,我没有自己的名字,张爱玲文集,色，戒,红楼梦魇,雷峰塔,传奇(上下),张看（上下）,传奇,﻿夹边沟记事,阿城精选集,额尔古纳河右岸,棋王.樹王.孩子王,写在人生边上 人生边上的边上 石语,人·兽·鬼,管锥编（全五册）,宋诗选注,钱锺书英文文集,钱钟书选集·散文卷，小说诗歌卷,钱锺书手稿集•外文笔记（第一辑）,钱钟书文集,钱钟书杨绛散文,傲慢与偏